In [1]:
import sys
from pathlib import Path

sys.path.append("../04_Src")

from services.forecasting_service import (
    load_lstm_model_and_metadata,
    load_latest_feature_window,
    predict_next_return_with_lstm,
)

project_root = Path("..").resolve()

model, meta = load_lstm_model_and_metadata(project_root=project_root)
window, last_date = load_latest_feature_window(
    project_root=project_root,
    window_size=meta["window_size"],
)
pred = predict_next_return_with_lstm(model, window)

print("Loaded model window_size:", meta["window_size"])
print("Last date:", last_date)
print("Predicted next day log return:", pred)


Loaded model window_size: 30
Last date: 2025-12-10 00:00:00
Predicted next day log return: -0.0008177310228347778


In [4]:
import sys
from pathlib import Path

sys.path.append("../04_Src")

from services.generative_service import (
    load_trained_vae,
    load_recent_return_window,
    generate_return_scenarios,
)

project_root = Path("..").resolve()

vae_model = load_trained_vae(project_root=project_root, seq_len=30)
window_returns, last_date = load_recent_return_window(
    project_root=project_root,
    window_size=30,
)

print("Recent window shape:", window_returns.shape)
print("Last date:", last_date)

scenarios = generate_return_scenarios(
    model=vae_model,
    last_window=window_returns,
    num_paths=10,
    noise_scale=1.0,
)
print("Scenarios shape:", scenarios.shape)



Loading VAE with latent_dim 12 hidden_dim 128
Recent window shape: (30,)
Last date: 2025-12-10 00:00:00
Scenarios shape: (10, 30)


In [5]:
import sys
from pathlib import Path

sys.path.append("../04_Src")

from services.explainability_service import (
    load_tabular_data,
    train_random_forest_for_explainability,
    compute_local_shap_for_index,
)

project_root = Path("..").resolve()

df_train_s, df_val_s, X_train_s, y_train_s, X_val_s, y_val_s, feature_cols_s = load_tabular_data(
    project_root=project_root
)

rf_expl = train_random_forest_for_explainability(X_train_s, y_train_s)

explanation = compute_local_shap_for_index(
    rf=rf_expl,
    df_val=df_val_s,
    X_val=X_val_s,
    feature_cols=feature_cols_s,
    index=len(df_val_s) - 1,
)

print("Date:", explanation["date"])
print("True:", explanation["true_target"])
print("Pred:", explanation["predicted_target"])
explanation["local_explanation"].head()


Date: 2023-12-29 00:00:00
True: -0.0056766607997231
Pred: -0.0020630480401129924


,feature,value,shap_value
0,log_return,-0.002830,0.000370
1,news_count,0.000000,0.000153
2,Open,4782.879883,0.000082
3,sentiment_pos_mean,0.000000,0.000000
4,sentiment_mean,0.000000,0.000000
